# Reading File

In [ ]:
def readFile(fileName):
    file = open(fileName, encoding = 'utf-8')
    rows = file.readlines() #read all lines
    file.close()

    file_ = []
    for row in rows:
        row = row.lower() #lower text
        file_.append(str(satir).rstrip("\n"))

    return file_

In [ ]:
files = readFile('hepsiS1.txt')
stopwords = readFile('turkish_stopwordsV3.txt')

# Word Count

In [ ]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
total_word = []
for txt in files:
  text_tokens = word_tokenize(txt)
  for token in text_tokens:
    total_word.append(token)
print("Toplam kelime sayısı: " + str(len(total_word)))
total_words = list(set(total_word))
print("Unique kelime sayısı: " + str(len(total_words)))

# Creating CSV files for training

In [ ]:
import csv
import nltk
from nltk.tokenize import word_tokenize
#create csv file
with open('hepsiS1.csv', 'w', newline='',encoding='utf-8') as file:
    fieldnames = ['labels', 'text']
    #define dictionary writer
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    ind = 0
    label = 0
    #read row in files
    for row in files:   
      #tokenize sentence
      text_tokens = word_tokenize(row)
      #remove stopwords in sentence
      tokens_without_sw = [word for word in text_tokens if not word in stopwords]
      #new sentence after processes
      filtered_sentence = (" ").join(tokens_without_sw)   
      writer.writerow({'labels': label, 'text': row})
      ind += 1
      if ind == 13676: #change label according to dataset's index
        label += 1

# Loading data for Turkish text classification and Create Dataframe

In [ ]:
import pandas as pd

df=pd.read_csv("hepsiS1.csv")
df.columns=["labels","text"]
df.labels=pd.Categorical(df.labels)
df['labels'] = pd.factorize(df.labels)[0]

In [ ]:
from simpletransformers.classification import ClassificationModel
import torch,sklearn
from sklearn.model_selection import train_test_split

# split a dataset into train and test sets
train_df, eval_df = train_test_split(df, test_size=0.2)

# Training Language Models

In [ ]:
modelb = ClassificationModel('bert', 'dbmdz/bert-base-turkish-cased', num_labels=2, 
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 3})

modelb.train_model(train_df, acc=sklearn.metrics.accuracy_score)

modele = ClassificationModel('electra', 'dbmdz/electra-base-turkish-cased-discriminator', num_labels=2, 
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 3})

modele.train_model(train_df, acc=sklearn.metrics.accuracy_score)

modela = ClassificationModel('albert', 'loodos/albert-base-turkish-uncased', num_labels=2, 
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 3})

modela.train_model(train_df, acc=sklearn.metrics.accuracy_score)

# Testing BERT Language Model

In [ ]:
test = eval_df 
result, model_outputs, wrong_predictions = modelb.eval_model(test)
predictions = model_outputs.argmax(axis=1)
actuals = test.labels.values
from sklearn.metrics import accuracy_score
print(accuracy_score(actuals, predictions))

Sentence predict example

In [ ]:
sample_text = test.iloc[10]['text']
print(sample_text)
model.predict([sample_text])
test.iloc[10]['labels']